In [1]:
import swarmrl as srl

from swarmrl.observables.top_down_image import TopDownImage
import numpy as np
import matplotlib.pyplot as plt
from swarmrl.components import Colloid
import open3d as o3d
import logging
import flax.linen as nn
from swarmrl.tasks.dummy_task import DummyTask
import optax
from swarmrl.actions.mpi_action import MPIAction
from swarmrl.engine.gaurav_sim import *
from swarmrl.trainers.global_continuous_trainer import GlobalContinuousTrainer as Trainer
import pint


2024-08-26 10:39:08.208362: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version (12.6.20). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
logging.basicConfig(level=logging.WARNING)

resolution=128

rafts = o3d.io.read_triangle_mesh("rafts.stl")

obs = TopDownImage(
    np.array([10000.0, 10000.0, 0.1]), image_resolution=np.array([resolution]*2), particle_type=0, custom_mesh=rafts, is_2D=True
)
task = DummyTask()


class ActoCriticNet(nn.Module):
    """A simple dense model."""

    @nn.compact
    def __call__(self, x):
        x = x.flatten()
        y = nn.Dense(features=2)(x)
        x = nn.Dense(features=2)(x)
        x = nn.relu(x)
        y = nn.relu(y)

        y = nn.Dense(features=12)(y)
        x = nn.Dense(features=12)(x)
        x = nn.relu(x)
        y = nn.relu(y)
        y = nn.Dense(features=1)(x)  # Critic
        x = nn.Dense(features=4)(x)  # Actor
        return x, y


exploration_policy = srl.exploration_policies.RandomExploration(probability=0.1)

# Define a sampling_strategy
sampling_strategy = srl.sampling_strategies.GumbelDistribution()

# Value function to use
value_function = srl.value_functions.ExpectedReturns(gamma=0.1, standardize=True)

# Define the model
actor_critic = ActoCriticNet()



[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x620f759c2de0 (threading is enabled)
EGL(1.5)
OpenGL(4.1)


In [3]:
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

# Define parameters in SI units
params = GauravSimParams(
            ureg=ureg,
            box_length=Q_(10000, "micrometer"),
            time_step=Q_(1e-3, "second"),
            time_slice=Q_(1, "second"),
            snapshot_interval=Q_(0.002, "second"),
            raft_radius=Q_(150, "micrometer"),
            raft_repulsion_strength=Q_(1e-7, "newton"),
            dynamic_viscosity=Q_(1e-3, "Pa * s"),
            fluid_density=Q_(1000, "kg / m**3"),
            lubrication_threshold=Q_(15, "micrometer"),
            magnetic_constant=Q_(4 * np.pi * 1e-7, "newton /ampere**2"),
            capillary_force_data_path=pathlib.Path(
                "/work/clohrmann/mpi_collab/capillaryForceAndTorque_sym6"
            ),  # TODO
        )

# Initialize the simulation system
system_runner = GauravSim(params=params, out_folder="./", with_precalc_capillary=True,save_h5=False)
mag_mom = Q_(1e-8, "ampere * meter**2")
for i in range(2):
    system_runner.add_colloids(pos = [np.random.rand()*10000,np.random.rand()*10000, 0]* ureg.micrometer, alpha = np.random.rand()*2*np.pi, magnetic_moment = 1E-8* ureg.ampere * ureg.meter**2)


# system_runner.add_colloids(Q_(np.array([300, 500]), "micrometer"), 0, mag_mom)
# system_runner.add_colloids(Q_(np.array([1000, 500]), "micrometer"), np.pi / 2, mag_mom)
# system_runner.add_colloids(
#     Q_(np.array([700, 1000]), "micrometer"), np.pi / 2 - 0.1, mag_mom
# )

In [4]:

network = srl.networks.ContinuousFlaxModel(
    flax_model=actor_critic,
    optimizer=optax.adam(learning_rate=0.01),
    input_shape=(resolution,resolution),
    sampling_strategy=sampling_strategy,
    exploration_policy=exploration_policy,
)
loss = srl.losses.PolicyGradientLoss(value_function=value_function)


actions = {
    0: MPIAction(
        amplitudes=np.array([0, 0]),
        phases=np.array([0, 0]),
        frequencies=np.array([0, 0]),
        offsets=np.array([0, 0]),
    ),
    1: MPIAction(
        amplitudes=np.array([0, 0]) ,
        phases=np.array([0, 0]) ,
        frequencies=np.array([0, 0]) ,
        offsets=np.array([0, 0]) ,
    ),
    2: MPIAction(
        amplitudes=np.array([0, 0]) ,
        phases=np.array([0, 0]) ,
        frequencies=np.array([0, 0]) ,
        offsets=np.array([0, 0]) ,
    ),
    3: MPIAction(
        amplitudes=np.array([0, 0]) ,
        phases=np.array([0, 0]) ,
        frequencies=np.array([0, 0]) ,
        offsets=np.array([0, 0]) ,
    ),
}
protocol = srl.agents.GlobalActorCriticAgent(
    particle_type=0,
    network=network,
    task=task,
    observable=obs,
    actions=actions,
    loss=loss,
)

# Define the force model.
rl_trainer = Trainer([protocol])


                             ActoCriticNet Summary                              
┏━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ path    ┃ module        ┃ inputs           ┃ outputs      ┃ params           ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│         │ ActoCriticNet │ float32[128,128] │ - float32[4] │                  │
│         │               │                  │ - float32[1] │                  │
├─────────┼───────────────┼──────────────────┼──────────────┼──────────────────┤
│ Dense_0 │ Dense         │ float32[16384]   │ float32[2]   │ bias: float32[2] │
│         │               │                  │              │ kernel:          │
│         │               │                  │              │ float32[16384,2] │
│         │               │                  │              │                  │
│         │               │                  │              │ 32,770 (131.1    │
│         │               │

In [5]:
rl_trainer.perform_rl_training(system_runner, 2, 10)

Output()

TypeError: translate(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cuda.pybind.geometry.Geometry3D, translation: numpy.ndarray[numpy.float64[3, 1]], relative: bool = True) -> open3d.cuda.pybind.geometry.Geometry3D

Invoked with: TriangleMesh with 443612 points and 201368 triangles., array([300., 500.])